In [2]:
!pip install -U langchain langchain_community openai streamlit transformers accelerate
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9

*1. Data Ingestion*

In [7]:
from langchain.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup
import requests


# Now fetch the content from the government site
gov_site_url = "https://immi.homeaffairs.gov.au/citizenship/become-a-citizen"
gov_response = requests.get(gov_site_url)
gov_soup = BeautifulSoup(gov_response.content, 'html.parser')

# Extract the content you need
content = "\n".join([p.get_text() for p in gov_soup.find_all("p")])

#add vector database


*2. Use openAI for Simplification and Summarization*

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain

llm = OpenAI(temperature=0.7, openai_api_key="")

# Define a template to simplify text
template = """
Rewrite the following content to follow the Australian Government Style Manual:
- Use plain English.
- Improve headings
Text: {content}
"""
# parse the style guide into it - headings improvement
prompt = PromptTemplate(template=template, input_variables=["content"])
# chain = LLMChain(llm=llm, prompt=prompt)
simplified_content = llm(prompt.format(content=content))
print(simplified_content)


Title: Guidelines for Writing in Plain English

Introduction

Writing in plain English is an essential skill for effectively communicating with your audience. It involves using simple, clear language that everyone can understand. The following guidelines will help you to write in plain English.

Why use plain English?

Plain English is important because it makes your message accessible to a wider audience. It removes confusion, reduces jargon, and ensures that your message is understood by all readers, regardless of their level of education or background.

Principles of plain English

To write in plain English, you should follow these principles:

1. Use short sentences: Short sentences are easier to read and understand.

2. Use everyday words: Avoid using technical jargon or complex language. Instead, use simple, everyday words that everyone can understand.

3. Be concise: Stick to the main points and avoid unnecessary details. This will make your writing more clear and to the point.

*3. Interactive Q&A System with LangChain Conversational Agents*

In [15]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)

# Simulate user query
user_query = "How can I apply for Australian citizenship?"
response = conversation.run(user_query)

print(response)


 Applying for Australian citizenship is a relatively straightforward process. First, you will need to determine if you are eligible for citizenship. This can depend on factors such as your current visa status, age, and length of time residing in Australia. Once you have confirmed your eligibility, you can fill out an online application through the Australian Department of Home Affairs website. The application will require you to provide personal information, such as your name, address, and date of birth. You will also need to provide evidence of your identity, such as a passport or birth certificate. Additionally, you will need to complete a citizenship test and attend an interview to assess your knowledge of Australian society, values, and customs. It is important to note that there is a fee for the application process, which can vary depending on your circumstances.


4. Another Agent as Senior Editor to verfiy if the generated content fit the style guide with measurement.
we can update the process by using GPT-3.5 or GPT-4 to handle the interactions between the Learner Agent and the Senior Editor Agent. Here’s how we can set up the system using GPT for both simplifying content and evaluating it for adherence to style guidelines.

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# Initialize GPT-3.5 or GPT-4 models for both Learner Agent and Senior Editor Agent
gpt_learner = OpenAI(model="gpt-3.5-turbo", openai_api_key="")  # Learner Agent (simplifies content)
gpt_editor = OpenAI(model="gpt-4", openai_api_key="")  # Senior Editor Agent (evaluates content)

# Define Technical Writer Agent (simplifies content)
def simplify_content(content):
    simplify_prompt = PromptTemplate(
        input_variables=["content"],
        template="Simplify the following content, such as improve headings for better understanding:\n{content}"
    )

    # chain = LLMChain(llm=gpt_learner, prompt=simplify_prompt)
    # simplified_content = chain.run(content)
    simplified_content = llm(simplify_prompt.format(content=content))
    return simplified_content

# Define Senior Editor Agent (evaluates simplified content)
def senior_editor(content, style_guide):
    style_prompt = PromptTemplate(
        input_variables=["content", "style_guide"],
        template=(
            "Review the following content according by an editor who has improved the headings "
            "Can you please provide some feedback:\n"
            "Content: {content}\n"
            "Style Guide: {style_guide}"
        )
    )

    # chain = LLMChain(llm=gpt_editor, prompt=style_prompt)
    # feedback = chain.run({"content": content, "style_guide": style_guide})
    feedback = llm(style_prompt.format(content=content, style_guide=style_guide))
    return feedback

# Sample Australian Style Manual guidelines
style_guide = """
Use headings help users scan content and find what they need.
"""

# Simulated workflow between Learner Agent and Senior Editor Agent
def collaborative_revision_process(content):
    # Step 1: Learner Agent generates simplified content
    simplified_content = simplify_content(content)
    print("Simplified Content:\n", simplified_content)

    # Step 2: Senior Editor Agent verifies and provides feedback
    feedback = senior_editor(simplified_content, style_guide)
    print("\n\nSenior Editor Feedback:\n", feedback)

    # Step 3: Learner Agent revises based on feedback (in real system, would resubmit)
    revised_content = simplified_content + " [Revision applied based on feedback]"

    return revised_content

# Example content to simplify from government site
gov_site_content = """
H1: Apply for a drivers licence
H2: Pass the theory test
H3: Learn the road rules
H3: Book a theory test
H2: Pass the driving test
H3: Practise for the test
H3: Book the driving test
"""

# Run the process
final_output = collaborative_revision_process(gov_site_content)


Simplified Content:
 
H1: How to Get a Driver's License
H2: Passing the Theory Test
H3: Understanding Road Rules
H3: Scheduling Your Theory Test
H2: Passing the Driving Test
H3: Preparing for the Test
H3: Scheduling Your Driving Test


Senior Editor Feedback:
 
Overall, the new headings are clear and organized. They follow a logical flow and provide specific information for each section. The use of subheadings under the main headings also helps break down the information into smaller chunks, making it easier for readers to digest. 

One suggestion for improvement would be to use consistent heading levels. For example, "Understanding Road Rules" is currently labeled as an H3, but it could be made into an H2 to match the other main headings. This would also create a clearer hierarchy and make the content easier to navigate.

Additionally, it would be helpful to include more descriptive headings. For instance, instead of just "Preparing for the Test," it could be expanded to "Preparing fo

*5.AutoGen Version*

In [1]:
!pip install openai==0.28
!pip install autogen -U

  Using cached openai-1.44.0-py3-none-any.whl.metadata (22 kB)
Using cached openai-1.44.0-py3-none-any.whl (367 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [2]:
from autogen import AutoGenAgent

# Define the agents
class LearnerAgent(AutoGenAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.role = "Learner Agent (Simplifier)"

    def perform_task(self, content):
        prompt = f"Simplify the following bureaucratic text into plain English:\n{content}"
        simplified_content = self.generate_response(prompt)
        return simplified_content


class SeniorEditorAgent(AutoGenAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.role = "Senior Editor Agent (Verifier)"

    def perform_task(self, content, style_guide):
        prompt = f"Verify the following simplified content against the style guide and provide feedback:\n"\
                 f"Content: {content}\nStyle Guide: {style_guide}"
        feedback = self.generate_response(prompt)
        return feedback


# Define style guide (you can extend this with detailed guidelines)
style_guide = """
1. Use plain English.
2. Write in an active voice.
3. Avoid unnecessary jargon.
4. Ensure inclusivity and accessibility.
"""

# Example government content
gov_content = """
Citizenship gives you the opportunity to participate in your community, vote, and feel a sense of belonging.
However, understanding the legal requirements of citizenship can be challenging due to complex language and bureaucratic processes.
"""

# Step 4: Set up communication between the agents
def simplified_content_workflow(gov_content, style_guide):
    # Initialize the learner agent and senior editor agent
    learner_agent = LearnerAgent(model="gpt-3.5-turbo")  # Learner agent using GPT-3.5 Turbo
    senior_editor_agent = SeniorEditorAgent(model="gpt-4")  # Senior Editor using GPT-4

    # Step 1: Learner agent simplifies the content
    simplified_content = learner_agent.perform_task(gov_content)
    print(f"Simplified Content:\n{simplified_content}\n")

    # Step 2: Senior Editor agent evaluates and provides feedback
    feedback = senior_editor_agent.perform_task(simplified_content, style_guide)
    print(f"Senior Editor Feedback:\n{feedback}\n")

    # Step 3: Learner agent revises the content based on feedback (optional, as a loop)
    revised_content = simplified_content + " [Revision applied based on feedback]"

    return revised_content

# Run the workflow
final_output = simplified_content_workflow(gov_content, style_guide)
print(f"Final Output:\n{final_output}")


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


ImportError: cannot import name 'AutoGenAgent' from 'autogen' (/usr/local/lib/python3.10/dist-packages/autogen/__init__.py)

*6. Combine with Vector Database*

In [20]:
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import faiss
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
import openai

# Set up OpenAI API Key
openai.api_key = ''

# Fetch the content from the government site
gov_site_url = "https://immi.homeaffairs.gov.au/citizenship/become-a-citizen"
gov_response = requests.get(gov_site_url)
gov_soup = BeautifulSoup(gov_response.content, 'html.parser')

# Extract the content you need
# The original code was only extracting text from <p> tags.
# This website has content in other tags like <h2>, <h3>, etc.
# The code below extracts text from all tags
content = "\n".join([tag.get_text() for tag in gov_soup.find_all()])

# Step 1: Convert the text into chunks for embedding
texts = content.split("\n")  # Simple chunking by paragraphs (or customize this)

# Check if any text was extracted
if not texts:
    raise ValueError("No content was extracted from the website.")

# Step 2: Initialize the embedding model
embedding_model = OpenAIEmbeddings(openai_api_key=openai.api_key)

# Step 3: Generate embeddings for the text chunks
embeddings = embedding_model.embed_documents(texts)

# Check if embeddings were generated
if not embeddings:
    raise ValueError("Embedding generation failed. No embeddings created.")

# Step 4: Initialize the FAISS index for vector storage
dimension = len(embeddings[0])  # Embedding vector dimension
index = faiss.IndexFlatL2(dimension)  # L2 distance metric

# Create a docstore for storing the texts
docstore = {i: text for i, text in enumerate(texts)}

# Create a mapping from FAISS index to docstore IDs
index_to_docstore_id = {i: i for i in range(len(texts))}

# Initialize FAISS with the docstore and index_to_docstore_id
faiss_index = FAISS(embedding_model.embed_query, index, docstore, index_to_docstore_id)

# Step 5: Add the document embeddings to FAISS vector store
faiss_index.add_texts(texts)

# Function to query FAISS vector store
def query_faiss_index(query):
    # Convert the query to an embedding vector
    query_embedding = embedding_model.embed_query(query)

    # Perform similarity search on the vector store
    results = faiss_index.similarity_search_with_score(query)

    # Output the top result's content
    top_result = results[0][0]
    return top_result

# Example: Querying content from the vector database, then processing it
query = "How to become an Australian citizen?"
gov_site_content = query_faiss_index(query)

# Define Learner Agent (simplifies content)
def simplify_content(content):
    simplify_prompt = PromptTemplate(
        input_variables=["content"],
        template="Simplify the following content, such as improve headings for better understanding:\n{content}"
    )

    chain = LLMChain(llm=gpt_learner, prompt=simplify_prompt)
    simplified_content = chain.run(content)
    return simplified_content

# Define Senior Editor Agent (evaluates simplified content)
def senior_editor(content, style_guide):
    style_prompt = PromptTemplate(
        input_variables=["content", "style_guide"],
        template=(
            "Review the following content according by an editor who has improved the headings "
            "Can you please provide some feedback:\n"
            "\nContent: {content}\nStyle Guide: {style_guide}"
        )
    )

    chain = LLMChain(llm=gpt_editor, prompt=style_prompt)
    feedback = chain.run({"content": content, "style_guide": style_guide})
    return feedback

# Simulated workflow between Learner Agent and Senior Editor Agent
def collaborative_revision_process(content):
    # Step 1: Learner Agent generates simplified content
    simplified_content = simplify_content(content)
    print("Simplified Content:\n", simplified_content)

    # Step 2: Senior Editor Agent verifies and provides feedback
    style_guide = "Use headings to help users scan content and find what they need."
    feedback = senior_editor(simplified_content, style_guide)
    print("\n\nSenior Editor Feedback:\n", feedback)

    # Step 3: Learner Agent revises based on feedback (in a real system, this would resubmit)
    revised_content = simplified_content + " [Revision applied based on feedback]"

    return revised_content

# Run the collaborative revision process on the retrieved content
final_output = collaborative_revision_process(gov_site_content)
print("\n\nFinal Revised Content:\n", final_output)
